In [ ]:
import os
import requests
import time
from tqdm import tqdm
from google.colab import drive

def get_drive_space():
    try:
        stats = os.statvfs("/content/drive")
        total = (stats.f_blocks * stats.f_frsize) / (2**30)
        used = ((stats.f_blocks - stats.f_bfree) * stats.f_frsize) / (2**30)
        free = (stats.f_bfree * stats.f_frsize) / (2**30)
        return f"Drive 용량: 총 {total:.1f}GB, 사용 {used:.1f}GB, 여유 {free:.1f}GB"
    except:
        return "Drive 용량 확인 실패"

def list_models(path):
    try:
        models = [f for f in os.listdir(path) if f.endswith(('.safetensors', '.ckpt', '.pt', '.pth'))]
        return "\n".join([f"- {model}" for model in models]) if models else "모델 파일 없음"
    except:
        return "모델 목록 확인 실패"

def download_model(url, path, api_token=""):
    try:
        model_id = url.split('/')[-1].split('?')[0]
        download_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor&size=pruned&fp=fp16"

        if api_token:
            download_url += f"&token={api_token}"

        response = requests.get(download_url, stream=True, allow_redirects=True)

        filename = response.headers.get('content-disposition', '').split('filename=')[-1].strip('"')
        if not filename:
            filename = f"model_{model_id}.safetensors"

        save_path = os.path.join(path, filename)
        total_size = int(response.headers.get('content-length', 0))

        print(f"📦 파일명: {filename}")
        print(f"📂 저장 경로: {save_path}")
        print(f"\n📥 다운로드 시작...")

        progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True,
                          desc="📊 진행 상황", mininterval=1.0, leave=False)
        start_time = time.time()

        with open(save_path, 'wb') as file:
            for data in response.iter_content(8192):
                if data:
                    file.write(data)
                    progress_bar.update(len(data))

        progress_bar.close()
        elapsed_time = time.time() - start_time
        print(f"\n✅ 다운로드 완료!")
        print(f"⏱️ 소요 시간: {elapsed_time:.2f}초")
        return True
    except Exception as e:
        print(f"\n❌ 다운로드 중 오류 발생: {e}")
        return False

mount_google_drive = True #@param {type:"boolean"}
force_remount = True #@param {type:"boolean"}
use_custom_path = False #@param {type:"boolean"}
custom_path = "" #@param {type:"string"}
download_url = "" #@param {type:"string"}
api_token = "" #@param {type:"string"}

if mount_google_drive:
    print("Google Drive를 마운트 중...")
    drive.mount('/content/drive', force_remount=force_remount)
    if use_custom_path and custom_path:
        download_path = os.path.join("/content/drive/MyDrive/", custom_path.strip("/"))
    else:
        download_path = "/content/drive/MyDrive/Civitai_Models/"
    print(get_drive_space())
else:
    if use_custom_path and custom_path:
        download_path = custom_path
    else:
        download_path = "/content/Civitai_Models/"

os.makedirs(download_path, exist_ok=True)

if download_model(download_url, download_path, api_token):
    print("\n📁 현재 모델 목록:")
    print(list_models(download_path))

if mount_google_drive:
    print("\n🔌 Google Drive 연결 해제 중...")
    try:
        drive.flush_and_unmount()
        print("✅ Google Drive 연결이 해제되었습니다.")
    except:
        print("❌ Google Drive 연결 해제 실패")